# Test McpExecutor Run Method

This notebook tests the `run` method of the `McpExecutor` class from `autocoder.common.mcp_tools`.

In [ ]:
import os
import sys
import json
import asyncio
import tempfile
from pathlib import Path

from autocoder.common.mcp_hub import McpHub, McpServer, McpTool, McpResource, McpResourceTemplate
from autocoder.common.mcp_tools import McpExecutor
import byzerllm

In [ ]:
# Helper function for async tests
async def run_test(test_func):
    """Helper to run async test functions"""
    try:
        await test_func()
        print("✅ Test passed")
    except AssertionError as e:
        print(f"❌ Test failed: {str(e)}")
    except Exception as e:
        print(f"❌ Test error: {str(e)}")

In [ ]:
async def test_mcp_executor_run():
    """Test McpExecutor run method"""
    # Create settings file with filesystem server config
    with tempfile.NamedTemporaryFile(mode='w', delete=False) as f:
        settings = {
            "mcpServers": {
                "filesystem": {
                    "command": "npx",
                    "args": [
                        "-y",
                        "@modelcontextprotocol/server-filesystem",
                        "/Users/allwefantasy/projects/tests"                        
                    ]
                }
            }
        }
        json.dump(settings, f)
        settings_path = f.name
    
    try:
        # Initialize hub and executor
        hub = McpHub(settings_path)
        await hub.initialize()
        
        llm = byzerllm.ByzerLLM.from_default_model(model="deepseek_chat")
        executor = McpExecutor(hub, llm)
        
        # Create test conversation
        conversations = [{
            "role": "user",
            "content": "List the contents of /Users/allwefantasy/projects/tests"
        }]
        
        # Run the executor
        result = await executor.run(conversations)
        
        # Verify the result contains the directory listing
        assert len(result) > 2, "Expected result to contain directory listing"
        print("Test result:", result[-2]['content'])
        
    finally:
        # Cleanup
        await hub.shutdown()
        os.unlink(settings_path)

In [ ]:
# Run the test
print("Testing McpExecutor run method:")
await run_test(test_mcp_executor_run)